In [23]:
import pypyodbc as odbc

# Correctly define variables
DRIVER_NAME = 'SQL SERVER'
SERVER_NAME = r'Pratiksharaut\SQLEXPRESS'  # raw string avoids \S escape issue
DATABASE_NAME = 'inventory'

# Properly format the connection string
connection_string = (
    f"DRIVER={{{DRIVER_NAME}}};"
    f"SERVER={SERVER_NAME};"
    f"DATABASE={DATABASE_NAME};"
    f"Trusted_Connection=yes;"1
)

# Connect

conn = odbc.connect(connection_string)
print(conn)

SyntaxError: invalid syntax. Perhaps you forgot a comma? (2011001739.py, line 10)

In [25]:
import pyodbc
from tkinter import messagebox

# Create the connection outside so you don't open/close repeatedly
conn = pyodbc.connect(
    'DRIVER={ODBC Driver 17 for SQL Server};'
    'SERVER=PRATIKSHARAUT\\SQLEXPRESS;'
    'DATABASE=inventory;'
    'Trusted_Connection=yes;'
)
cursor = conn.cursor()

In [27]:
from tkinter import *
import pyodbc as odbc
import matplotlib.pyplot as plt
from tkinter import Tk, Label
from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg
from tkinter import ttk
from tkinter import messagebox
from tkcalendar import DateEntry
from datetime import datetime


In [29]:
import tkinter as tk
from tkinter import ttk, Frame, Label, Button, Entry, Checkbutton, IntVar, LabelFrame, messagebox, StringVar
from tkinter.constants import X, BOTH, LEFT, GROOVE, RAISED, W
from tkcalendar import DateEntry
from statistics import mode, StatisticsError
current_row = 0
list_price_entries = []

def Bill_entry():
    root = tk.Tk()
    root.geometry("1350x750+0+0")
    root.title("Billing Management System")

    top_frame = Frame(root, bg="#0f4d7d", bd=7, relief=GROOVE)
    top_frame.pack(fill=X)

    Button(top_frame, text="← Back", font=("Arial", 12, "bold"), bg="white", fg="#0f4d7d",
           bd=3, relief=RAISED, command=root.destroy).pack(side=LEFT, padx=10, pady=5)

    Label(top_frame, text="Billing Entry", font=('Times new roman', 30, "bold"),
          bg="#0f4d7d", fg="white").pack(side=LEFT, padx=20)

    Billing_frame = Frame(root, bg="white", bd=5, relief=GROOVE)
    Billing_frame.pack(fill=BOTH, expand=True)

    F1 = LabelFrame(Billing_frame, text="Customer Details", bd=5, relief=GROOVE,
                    font=('times new roman', 15, "bold"), bg="#0f4d7d", fg="gold")
    F1.place(x=0, y=0, relwidth=1, height=120)

    name_var = tk.StringVar()
    phone_var = tk.StringVar()
    billno_var = tk.StringVar()
    address_var = tk.StringVar()

   
    Label(F1, text="Customer Name:", font=('times new roman', 15, "bold"), bg="#0f4d7d", fg="gold")\
        .grid(row=0, column=0, padx=20, pady=5)
    Entry(F1, font=("arial", 15), bd=3, relief=GROOVE, textvariable=name_var).grid(row=0, column=1, padx=10, pady=5)

    Label(F1, text="Phone No.:", font=('times new roman', 15, "bold"), bg="#0f4d7d", fg="gold")\
        .grid(row=0, column=2, padx=20, pady=5)
    Entry(F1, font=("arial", 15), bd=3, relief=GROOVE, textvariable=phone_var).grid(row=0, column=3, padx=10, pady=5)

    Label(F1, text="Bill Number:", font=('times new roman', 15, "bold"), bg="#0f4d7d", fg="gold")\
        .grid(row=0, column=4, padx=20, pady=5)
    Entry(F1, font=("arial", 15), bd=3, relief=GROOVE, textvariable=billno_var).grid(row=0, column=5, padx=10, pady=5)

    # Your existing fetch_customer_details with minor adjustment
    def fetch_customer_details():
        billno = billno_var.get().strip()
        customer_name = name_var.get().strip()

        if not billno and not customer_name:
            messagebox.showwarning("Input Error", "Please enter either Bill Number or Customer Name")
            return

        try:
            found = False

            if billno:
                cursor.execute("SELECT CustomerName, PhoneNo, Address FROM Customers WHERE BillNumber = ?", (billno,))
                row = cursor.fetchone()
                if row:
                    name_var.set(row[0])
                    phone_var.set(row[1])
                    address_var.set(row[2])
                    found = True

            if not found and customer_name:
                cursor.execute("SELECT BillNumber, PhoneNo, Address FROM Customers WHERE LOWER(CustomerName) LIKE ?", (f"%{customer_name.lower()}%",))
                row = cursor.fetchone()
                if row:
                    billno_var.set(row[0])
                    phone_var.set(row[1])
                    address_var.set(row[2])
                    name_var.set(customer_name)
                    found = True

            if not found:
                messagebox.showerror("Not Found", "No customer found with the given details.")
                billno_var.set("")
                name_var.set("")
                phone_var.set("")
                address_var.set("")

        except Exception as e:
            messagebox.showerror("Database Error", f"Error fetching data: {e}")

    Button(F1, text="Search", font=("Arial", 12, "bold"), bg="white", fg="#0f4d7d",
           command=fetch_customer_details).grid(row=0, column=6, padx=10, pady=5)

    # --- Live Search Popup for Customer Name ---
    def show_customer_suggestions(event):
        typed = name_var.get().strip()
        if not typed:
            if hasattr(root, 'cust_popup'):
                root.cust_popup.destroy()
            return

        try:
            # Search customers starting with typed letters (case-insensitive)
            cursor.execute("SELECT CustomerName FROM Customers WHERE LOWER(CustomerName) LIKE ? ORDER BY CustomerName ASC", (typed.lower() + '%',))
            results = cursor.fetchall()
            names = [row[0] for row in results]

            if not names:
                if hasattr(root, 'cust_popup'):
                    root.cust_popup.destroy()
                return

            # Destroy old popup if exists
            if hasattr(root, 'cust_popup'):
                root.cust_popup.destroy()

            popup = tk.Toplevel(root)
            root.cust_popup = popup
            popup.wm_overrideredirect(True)  # No window decorations
            x = name_entry.winfo_rootx()
            y = name_entry.winfo_rooty() + name_entry.winfo_height()
            popup.geometry(f"+{x}+{y}")

            listbox = tk.Listbox(popup, font=("arial", 12), width=30, height=min(len(names), 10))
            listbox.pack()

            for name in names:
                listbox.insert(tk.END, name)

            def on_select(event):
                if listbox.curselection():
                    selected_name = listbox.get(listbox.curselection())
                    name_var.set(selected_name)

                    # Fetch details for selected customer name
                    try:
                        cursor.execute("SELECT BillNumber, PhoneNo, Address FROM Customers WHERE CustomerName = ?", (selected_name,))
                        row = cursor.fetchone()
                        if row:
                            billno_var.set(row[0])
                            phone_var.set(row[1])
                            address_var.set(row[2])
                        else:
                            billno_var.set("")
                            phone_var.set("")
                            address_var.set("")
                    except Exception as e:
                        messagebox.showerror("Database Error", f"Error fetching data: {e}")

                    popup.destroy()
                    name_entry.focus_set()

            listbox.bind("<<ListboxSelect>>", on_select)

            # Close popup when focus lost
            popup.bind("<FocusOut>", lambda e: popup.destroy())

        except Exception as e:
            if hasattr(root, 'cust_popup'):
                root.cust_popup.destroy()

    name_var = tk.StringVar()
    # Define name_entry BEFORE you bind events to it
    name_entry = Entry(F1, font=("arial", 15), bd=3, relief=GROOVE, textvariable=name_var)
    name_entry.grid(row=0, column=1, padx=10, pady=5)
    
    # Then bind the event after name_entry is defined
    name_entry.bind("<KeyRelease>", show_customer_suggestions)



    Button(F1, text="Search", font=("Arial", 12, "bold"), bg="white", fg="#0f4d7d",
           command=fetch_customer_details).grid(row=0, column=6, padx=10, pady=5)

    Label(F1, text="Shipping Address:", font=('times new roman', 15, "bold"), bg="#0f4d7d", fg="gold")\
        .grid(row=1, column=0, padx=5, pady=5, sticky=W)
    Entry(F1, font=("arial", 15), bd=3, relief=GROOVE, textvariable=address_var)\
        .grid(row=1, column=1, padx=5, pady=5)

    Label(F1, text="Bill Date:", font=('times new roman', 15, "bold"), bg="#0f4d7d", fg="gold")\
        .grid(row=1, column=2, padx=5, pady=5)
    DateEntry(F1, font=("arial", 15), bd=3, relief=GROOVE, background='darkblue',
              foreground='white', date_pattern='dd/mm/yyyy')\
        .grid(row=1, column=3, padx=5, pady=5, sticky=W)

    Label(F1, text="Delivery Date:", font=('times new roman', 15, "bold"), bg="#0f4d7d", fg="gold")\
        .grid(row=1, column=4, padx=5, pady=5)
    DateEntry(F1, font=("arial", 15), bd=3, relief=GROOVE, background='darkblue',
              foreground='white', date_pattern='dd/mm/yyyy')\
        .grid(row=1, column=5, padx=5, pady=5, sticky=W)
     
    def check_customer_details_filled():
            name = name_var.get().strip()
            contact = contact_var.get().strip()
            address = address_var.get().strip()
        
            missing_fields = []
            if not name:
                missing_fields.append("Name")
            if not contact:
                missing_fields.append("Contact")
            if not address:
                missing_fields.append("Address")
        
            if missing_fields:
                messagebox.showerror(
                    "Missing Customer Details",
                    f"Please fill in the following fields before proceeding: {', '.join(missing_fields)}"
                )
                return False
            return True
    Label(F1, text="Cash Disc:", font=('times new roman', 15, "bold"), bg="#0f4d7d", fg="gold")\
        .grid(row=1, column=6, padx=5)
    Entry(F1, font=("arial", 15), bd=3, relief=GROOVE, width=4).grid(row=1, column=7, padx=5, sticky=W)

    def on_discount_enter(event, row):
        try:
            discount_text = row[7].get().strip()
            qty = float(row[4].get())
            price = float(row[6].get())
    
            if discount_text.endswith('%'):
                discount_val = float(discount_text[:-1])
                unit_price = price - (price * discount_val / 100)
            else:
                discount_val = float(discount_text)
                unit_price = price - discount_val
    
            # Update DISC UNIT PRICE (column 8)
            row[8].config(state="normal")
            row[8].delete(0, tk.END)
            row[8].insert(0, f"{unit_price:.2f}")
            row[8].config(state="readonly")
    
            # Update DISC GROSS PRICE (column 9)
            gross_price = unit_price * qty
            row[9].config(state="normal")
            row[9].delete(0, tk.END)
            row[9].insert(0, f"{gross_price:.2f}")
            row[9].config(state="readonly")
    

    
        except Exception as e:
            print(f"Error in discount entry: {e}")
     # ---------- Row Logic ----------
    entry_rows = []
    current_row = 1
    
    def update_lp_total():
        total = 0.0
        for row in entry_rows:
            val = row[10].get()
            try:
                if val:
                    total += float(val)
            except ValueError:
                pass
        lp_total_var.set(f"{total:.2f}")
    
    def lp_price(event, row):
        try:
            qty = float(row[4].get())
            price = float(row[6].get())
    
            if qty < 0 or price < 0:
                raise ValueError("Quantity and Price must be non-negative.")
    
            total_price = qty * price
            row[10].config(state="normal")
            row[10].delete(0, tk.END)
            row[10].insert(0, f"{total_price:.2f}")
            row[10].config(state="readonly")
    
        except ValueError as ve:
            row[10].config(state="normal")
            row[10].delete(0, tk.END)
            row[10].insert(0, "")
            row[10].config(state="readonly")
         
        update_lp_total()

    def apply_cash_discount():
        try:
            discount_val = float(cash_disc_var.get())
        except ValueError:
            messagebox.showerror("Invalid Input", "Enter a valid cash discount.")
            return
    
        is_percent = cash_chk_var.get() == 1
        total_gross_price = 0
    
        # Calculate total gross price (sum of all row gross prices)
        for row in entry_rows:
            try:
                gross = float(row[9].get())  # Assuming row[9] holds the gross price of that row
                total_gross_price += gross
            except Exception:
                continue
    
        # Apply cash discount on overall total gross price
        if is_percent:
            discounted_total = total_gross_price * (1 - discount_val / 100)
        else:
            discounted_total = total_gross_price - discount_val
            if discounted_total < 0:
                discounted_total = 0  # Avoid negative total
    
        # Update net discounted total price in F3 Entry
        net_disc_price_var.set(f"₹ {discounted_total:.2f}")

        """
    def update_total_price():
            total = 0
            for row in entry_rows:
                try:
                    gross = float(row[9].get())
                    total += gross
                except:
                    continue
            lp_total_var.set(f"₹ {total:.2f}")
        
        
        """      
        
    
        


    Button(F1, text="Appl ✅", font=("Arial", 10, "bold"), bg="white", fg="#0f4d7d",
       width=10, anchor="center", command=apply_cash_discount).grid(row=1, column=8, padx=5)


    cash_disc_var = StringVar(value="0")
    cash_chk_var = IntVar()
    Entry(F1, textvariable=cash_disc_var, font=("arial", 15), bd=3, relief=GROOVE, width=4).grid(row=1, column=7, padx=5, sticky=W)
    
    Checkbutton(F1, text="%", variable=cash_chk_var, bg="#0f4d7d", font=("Arial", 18, "bold"),
                activebackground="white", fg="black", selectcolor="white").grid(row=1, column=8, sticky=W)

    F2 = LabelFrame(Billing_frame, bd=5, relief=GROOVE, font=('times new roman', 15, "bold"),
                    bg="white", fg="gold")
    F2.place(x=0, y=110, relwidth=1, height=450)
    
    canvas = tk.Canvas(F2, bg="white")
    canvas.pack(side=tk.TOP, fill=BOTH, expand=True)
    
    h_scrollbar = tk.Scrollbar(F2, orient=tk.HORIZONTAL, command=canvas.xview)
    h_scrollbar.pack(side=tk.BOTTOM, fill=X)
    
    canvas.configure(xscrollcommand=h_scrollbar.set)
    
    scrollable_frame = Frame(canvas, bg="white")
    canvas.create_window((0, 0), window=scrollable_frame, anchor="nw")
    
    def on_configure(event):
        canvas.configure(scrollregion=canvas.bbox("all"))
    
    scrollable_frame.bind("<Configure>", on_configure)
    
    headers = [
    "DESCRIPTION", "RATINGS", "CAT NO.", "MAKE", "QUANTITY",
    "NET QTY", "LIST PRICE", "DISCOUNT", "DISC UNIT PRICE", "DISC GROSS PRICE","LP GRASS.PRICE"
    ]

    for col, text in enumerate(headers):
        lbl = Label(scrollable_frame, text=text, font=('times new roman', 13),
                    bg="#A1AEB1", fg="black", width=20)
        lbl.grid(row=0, column=col, padx=1, pady=0, sticky='w')
    
    # The big Entry spanning all columns
    B1 = tk.Entry(scrollable_frame, font=('times new roman', 12, "bold"),
                  bg="#DBE2E9", fg="black", width=100)
    B1.grid(row=1, column=0, columnspan=len(headers), sticky='we', padx=1, pady=0)

    # Configure grid column 0 to expand, others not
    scrollable_frame.grid_columnconfigure(0, weight=1)
    for col in range(1, len(headers)):
        scrollable_frame.grid_columnconfigure(col, weight=0)
  
    # Entry widgets in row 2
    entries = []
    for col in range(len(headers)):
        entry = tk.Entry(scrollable_frame, font=("arial", 12), bd=3, relief=tk.GROOVE, width=20)
        entry.grid(row=2, column=col, padx=1, pady=1)
        entries.append(entry)
    
    entry_rows = []
    current_row = 2  # After header row and B1

    def on_qty_enter(event, row_entries):
        # Only add a new row if this is the last one
        if row_entries == entry_rows[-1]:
            add_entry_row()

    def add_section_heading_row():
        nonlocal current_row
        
        # Add a big Entry spanning all columns like B1
        section_entry = tk.Entry(scrollable_frame, font=('times new roman', 12, "bold"),
                                 bg="#DBE2E9", fg="gold", width=100)
        section_entry.grid(row=current_row, column=0, columnspan=len(headers), sticky='we', padx=1, pady=0)
        
        # Bind Enter key to add normal entry row below this heading
        def on_section_enter(event):
            if section_entry.get().strip() != "":
                add_entry_row()
                section_entry.config(state='disabled')  # Disable editing once section heading is entered
                section_entry.unbind("<Return>")  # Remove binding after use
            return "break"  # Prevent default behavior
        
        section_entry.bind("<Return>", on_section_enter)
        
        current_row += 1

    def on_qty_enter(event, row_entries):
        # Only add a new row if this is the last one
        if row_entries == entry_rows[-1]:
            qty_val = row_entries[4].get().strip()
            if qty_val == "":
                # Add section heading row instead of normal entry row
                add_section_heading_row()
            else:
                add_entry_row()
    
    def add_entry_row():
        nonlocal current_row
    
        row_entries = []
        readonly_columns = (8, 9, 10)  # DISC UNIT, DISC GROSS, LP GROSS.PRICE
    
        for col in range(len(headers)):
            state = "readonly" if col in readonly_columns else "normal"
            e = tk.Entry(scrollable_frame, font=("arial", 12), bd=3,
                         relief=tk.GROOVE, width=20, state=state, fg="black")
            e.grid(row=current_row, column=col, padx=1, pady=1)
            row_entries.append(e)
    
        # Bind popup to DESCRIPTION
        row_entries[0].bind("<KeyRelease>", lambda event, r=row_entries: show_popup(event, r))
    
        # QUANTITY (index 4) — bind both Enter and FocusOut
        row_entries[4].bind("<Return>", lambda event, r=row_entries: on_qty_enter(event, r))
        row_entries[4].bind("<FocusOut>", lambda event, r=row_entries: lp_price(event, r))
    
        # LIST PRICE (index 6) — bind FocusOut for LP calculation
        row_entries[6].bind("<FocusOut>", lambda event, r=row_entries: lp_price(event, r))
    
        # DISCOUNT (index 7) — bind Enter key
        row_entries[7].config(state="normal")
        row_entries[7].bind("<Return>", lambda event, r=row_entries: on_discount_enter(event, r))
    
        entry_rows.append(row_entries)
        current_row += 1
    

    inventory = [  # Add this dummy inventory list
    {"description": "MCCB 100A", "make": "ABB", "qty": 5, "ratings": "100A", "catno": "ABB100A", "listprice":26710},
    {"description": "MCB 32A", "make": "Siemens", "qty": 10, "ratings": "32A", "catno": "SIE32A", "listprice":400},
    {"description": "Contactors 9A", "make": "Schneider", "qty": 12, "ratings": "9A", "catno": "SCH9A", "listprice":750},
    {"description": "TPN DB 4WAY", "make": "Legrand", "qty": 3, "ratings": "4W", "catno": "LEG4W", "listprice":50},
    {"description": "Cable 10sqmm", "make": "Polycab", "qty": 20, "ratings": "10sqmm", "catno": "POL10SQ", "listprice":200},
    {"description": "ELCB 40A", "make": "Anchor", "qty": 60, "ratings": "40A","catno": "ELCB40", "listprice":680},
    ]

    def show_popup(event, entry_list):
        typed = entry_list[0].get().lower()
        if not typed:
            return
    
        matches = [item for item in inventory if item["description"].lower().startswith(typed)]
        if not matches:
            return
    
        popup = tk.Toplevel()
        popup.wm_title("Select Item")
        popup.geometry("550x200+%d+%d" % (entry_list[0].winfo_rootx(), entry_list[0].winfo_rooty() + 30))
        popup.grab_set()
    
        tree = ttk.Treeview(popup, columns=("description", "make", "qty", "ratings", "catno", "listprice","net_qty"), show="headings")
        tree.heading("description", text="DESCRIPTION")
        tree.heading("make", text="MAKE")
        tree.heading("qty", text="QTY")
        tree.heading("ratings", text="RATINGS")
        tree.heading("catno", text="CAT NO.")
        tree.heading("listprice",text="LIST PRICE")
    
    
        for item in matches:
            tree.insert("", "end", values=(
                item["description"], item["make"], item["qty"], item["ratings"], item["catno"], item["listprice"]
            ))

                # Calculate mode of qty for net_qty column
        from statistics import mode, StatisticsError
        try:
            qty_mode = mode([item["qty"] for item in inventory])
        except StatisticsError:
            qty_mode = ""  # Handle case with no unique mode
        

        for item in matches:
            tree.insert("", "end", values=(
                item["description"],
                item["make"],
                item["qty"],
                item["ratings"],
                item["catno"],
                item["listprice"],
                qty_mode   # 👈 This sets NET_QTY as the mode of qty
            ))
        
            
        tree.pack(fill="both", expand=True)
    
        def on_select(event):
            selected = tree.focus()
            values = tree.item(selected, "values")
            if values:
                entry_list[0].delete(0, tk.END)
                entry_list[0].insert(0, values[0])  # DESCRIPTION
    
                entry_list[1].delete(0, tk.END)
                entry_list[1].insert(0, values[3])  # RATINGS
    
                entry_list[2].delete(0, tk.END)
                entry_list[2].insert(0, values[4])  # CAT NO.
    
                entry_list[3].delete(0, tk.END)
                entry_list[3].insert(0, values[1])  # MAKE
    
                entry_list[4].delete(0, tk.END)
                entry_list[4].insert(0, values[2])  # QTY

                entry_list[5].delete(0, tk.END)
                entry_list[5].insert(0, values[2]) #listprice

                entry_list[6].delete(0, tk.END)
                entry_list[6].insert(0, values[5])

                
    
                popup.destroy()
    
        tree.bind("<Double-1>", on_select)
    
    # Make sure this is after `entries` is created and `show_popup` is defined
    entries[0].bind("<KeyRelease>", lambda event: show_popup(event, entries))

        
    try:
        qty_mode = mode([item["qty"] for item in inventory])
    except StatisticsError:
        qty_mode = ""  # if no unique mode exists
    
    F3 = LabelFrame(Billing_frame, text="Bill Menu", bd=5, relief=GROOVE,
                    font=('times new roman', 15, "bold"), bg="#0f4d7d", fg="gold")
    F3.place(x=0, y=560, relwidth=1, height=160)
    
    enclosure_total_var = tk.StringVar(value="₹ 0.00")
    def open_enclosure_popup(total_var=None):
        popup = tk.Toplevel()
        popup.title("Enclosure Fabrication")
        popup.geometry("1100x700")
        popup.configure(bg="white")
    
        headers = ["✔"," ", "MM", "Length", "Width", "", "Kg", "", "Kg per Sqmm"]
    
        enclosure_data = [
            {" ": "18 SWG", "MM": 1.20, "Length": 2500, "Width": 1250, "Kg": 30, "": 29.4375, "Kg per Sqmm": 0.0000096},
            {" ": "16 SWG", "MM": 1.50, "Length": 2500, "Width": 1250, "Kg": 40, "": 36.796875, "Kg per Sqmm": 0.0000128},
            {" ": "14 SWG", "MM": 2.00, "Length": 2500, "Width": 1250, "Kg": 50, "": 49.0625, "Kg per Sqmm": 0.000016},
            {" ": "13 SWG", "MM": 2.30, "Length": 2500, "Width": 1250, "Kg": 60, "": 56.421875, "Kg per Sqmm": 0.0000192},
            {" ": "12 SWG", "MM": 2.64, "Length": 2500, "Width": 1250, "Kg": 70, "": 64.7625, "Kg per Sqmm": 0.0000224},
            {" ": "11 SWG", "MM": 3.00, "Length": 2500, "Width": 1250, "Kg": 80, "": 73.59375, "Kg per Sqmm": 0.0000256},
            {" ": "10 SWG", "MM": 3.25, "Length": 2500, "Width": 1250, "Kg": 90, "": 79.7265625, "Kg per Sqmm": 0.0000288},
        ]
    
        entry_matrix1 = []
        checkbox_vars = []
    
        def update_total(row_idx):
            try:
                mm = float(entry_matrix1[row_idx][1].get())
                length = float(entry_matrix1[row_idx][2].get())
                width = float(entry_matrix1[row_idx][3].get())
                total = round(mm * length * width * 0.00000785, 2)
                entry_matrix1[row_idx][4].config(state='normal')
                entry_matrix1[row_idx][4].delete(0, tk.END)
                entry_matrix1[row_idx][4].insert(0, total)
                entry_matrix1[row_idx][4].config(state='readonly', disabledforeground="black")
            except ValueError:
                pass
    
        def update_kg_per_sqmm(row_idx):
            try:
                kg = float(entry_matrix1[row_idx][5].get())
                length = float(entry_matrix1[row_idx][2].get())
                width = float(entry_matrix1[row_idx][3].get())
                if length * width == 0:
                    return
                result = (kg / (length * width))
                entry_matrix1[row_idx][7].config(state='normal')
                entry_matrix1[row_idx][7].delete(0, tk.END)
                sqr_meter="{:.10f}".format(result)
                entry_matrix1[row_idx][7].insert(0,sqr_meter )
                entry_matrix1[row_idx][7].config(state='readonly', disabledforeground="black")
            except ValueError:
                pass
        popup.grid_columnconfigure(0, minsize=20, weight=0)  # 40 pixels ≈ 4 character
        # Header labels
        for col, text in enumerate(headers):
            tk.Label(popup, text=text if text else "Calculated", font=('times new roman', 12, "bold"),
                     bg="#A1AEB1", fg="black", width=20).grid(row=0, column=col, padx=1, pady=10)
            tk.Label(popup, bg="#A1AEB1", width=2).grid(row=0, column=0, padx=1, pady=10)
        for row_index, data in enumerate(enclosure_data):
            row_entries = []
            var = tk.IntVar()
            checkbox_vars.append(var)  # Store the checkbox variable
        
            # Add checkbox in the first column
            chk = tk.Checkbutton(popup, variable=var, bg="white", padx=0, pady=0, anchor="center", takefocus=0)
            chk.grid(row=row_index + 1, column=0, padx=1, pady=1, sticky="w")
            row_entries.append(None)  # Placeholder to keep matrix aligned
        
            for col_index, header in enumerate(headers[1:], start=1):  # Start from column 1
                ent = tk.Entry(popup, font=('times new roman', 12), width=20)
                ent.grid(row=row_index + 1, column=col_index, padx=1, pady=1)
                popup.grid_columnconfigure(0, minsize=30)
        
                # Insert values or calculated fields
                if header in data:
                    ent.insert(0, data[header])
                elif col_index == 4:  # Column for MM*L*W*0.00000785
                    mm = float(data.get("MM", 0))
                    length = float(data.get("Length", 0))
                    width = float(data.get("Width", 0))
                    total = round(mm * length * width * 0.00000785, 2)
                    ent.insert(0, total)
                    ent.config(state='readonly', disabledforeground="black")
                elif col_index == 7:  # Column for Kg/(L*W)
                    kg = float(data.get("Kg", 0))
                    length = float(data.get("Length", 0))
                    width = float(data.get("Width", 0))
                    if length * width > 0:
                        result = (kg / (length * width))
                        sqr_meter = "{:.10f}".format(result)
                        ent.insert(0, sqr_meter)
                        ent.config(state='readonly', disabledforeground="black")
        
                # Dynamic bindings
                if col_index in [1, 2, 3]:
                    ent.bind("<FocusOut>", lambda e, r=row_index: update_total(r))
                    ent.bind("<Tab>", lambda e, r=row_index: update_total(r))
        
                if col_index in [2, 3, 5]:
                    ent.bind("<FocusOut>", lambda e, r=row_index: update_kg_per_sqmm(r))
                    ent.bind("<Tab>", lambda e, r=row_index: update_kg_per_sqmm(r))
        
                if row_index in [0, 2, 4, 6]:
                    ent.config(bg="#FF7F7F")
        
                row_entries.append(ent)
            entry_matrix1.append(row_entries)

        headers1 = ["", "L", "W", "H", " "]  # Last column is extra (e.g., QTY)
        second_table_row_start = len(enclosure_data) + 3
        column_width = 20

        # Header color
        header_bg = "#FFD700"
        header_fg = "#4B4B4B"

        # Initial data for first row (row 0)
        Enclosure_data1 = [
            {"": "28+1+1", "L": 2400, "W": 300, "H": 1600, " ": ""},
        ]

        # Define autofill behavior (L and W only)
        autofill_map = {
            1: [1, 3],  # L column
            2: [3],     # W column
            # H and last column are manual
        }


        # Fallback values if target cell is empty
        fallback_values = {
            (2, 1): 300,      # row 2, col 1 (L)
            (1, 2): 1600,     # row 1, col 2 (W)
            (2, 2): 1600,     # row 2, col 2 (W)
        }

        # Entry matrix for accessing any cell later
        entry_matrix2 = []

        # Smart autofill function
        def smart_autofill(col_index, value):
            if col_index == 3 or col_index == 4:  # Skip autofill for H and last column
                return
        
            filled_rows = set()
        
            # Autofill to specific mapped rows
            for row in autofill_map.get(col_index, []):
                entry_matrix2[row][col_index].delete(0, tk.END)
                entry_matrix2[row][col_index].insert(0, value)
                filled_rows.add(row)
        # Apply fallback if cell is still empty
            for row in range(1, 5):  # Skip row 0
                if (row, col_index) not in filled_rows:
                    cell = entry_matrix2[row][col_index]
                    if not cell.get().strip():
                        fallback = fallback_values.get((row, col_index))
                        if fallback is not None:
                            cell.insert(0, fallback)

        # Create header row
        for col, text in enumerate(headers1):
             tk.Label(popup, text=text, font=('times new roman', 12, "bold"),
                     bg=header_bg, fg=header_fg, width=column_width).grid(
                     row=second_table_row_start, column=col, padx=1, pady=(15, 0))  # ⬅️ Adds top spacing only               
                    # Create entry grid
        for row in range(7):  # 5 rows (0 for input, 1–4 for fill/serials)
            row_entries = []
            for col, header in enumerate(headers1):
                bg_color = "#FFD700" if row == 0 else "white"
                ent = tk.Entry(popup, font=('times new roman', 12), bg=bg_color, width=column_width)
                ent.grid(row=second_table_row_start + row + 1, column=col, padx=1, pady=1)
        
                # Row 0 gets initial values
                if row == 0:
                    ent.insert(0, Enclosure_data1[0].get(header, ""))
        
                    def make_callback(c=col):
                        def on_focus_out(event):
                            value = event.widget.get()
                            smart_autofill(c, value)
                        return on_focus_out
                    ent.bind("<FocusOut>", make_callback())
        # Column 0 → Serial Numbers for rows 1 to 3
                elif col == 0 and 1 <= row <= 6:
                    ent.insert(0, str(row))  # 1, 2, 3
        
                row_entries.append(ent)
            entry_matrix2.append(row_entries)
        # ---------- Total Calculation ----------
        def enclosure_calculate_total():
            total_volume = 0
            for row in entry_matrix2[1:7]:  # Skip header/input row
                try:
                    l = float(row[1].get())  # L
                    w = float(row[2].get())  # W
                    h = float(row[3].get())  # H
                    volume = l * w * h
                    total_volume += volume
        
                    # Show volume in last column
                    row[4].delete(0, tk.END)
                    row[4].insert(0, f"{int(volume)}")
                except ValueError:
                    row[4].delete(0, tk.END)

            enclosure_total_var.set(f"{int(total_volume)}")
        def handle_percentage_focusout(event):
            enclosure_calculate_total()
            widget = event.widget
            for i, row in enumerate(entry_matrix2):
                if widget in row:
                    current_row = i
                    break
            else:
                return  # not found
        
            try:
                percent_text = entry_matrix2[current_row][3].get().strip()
                if percent_text.endswith("%"):
                    percent = float(percent_text[:-1])
                else:
                    percent = float(percent_text)
        
                total_volume = float(enclosure_total_var.get())
                result = (percent / 100) * total_volume
                final = total_volume + result
        
                entry_matrix2[current_row][4].delete(0, tk.END)
                entry_matrix2[current_row][4].insert(0, f"{round(result, 2)}")
        
                # Add a 6th column dynamically if needed
                if len(entry_matrix2[current_row]) <= 5:
                    ent = tk.Entry(popup, font=('times new roman', 12), width=column_width)
                    ent.grid(row=second_table_row_start + current_row + 1, column=5, padx=1, pady=1)
                    entry_matrix2[current_row].append(ent)
                else:
                    ent = entry_matrix2[current_row][5]
        
                ent.delete(0, tk.END)
                ent.insert(0, f"{round(final, 2)}")
        
            except Exception:
                entry_matrix2[current_row][4].delete(0, tk.END)
                if len(entry_matrix2[current_row]) > 5:
                    entry_matrix2[current_row][5].delete(0, tk.END)
        
        # Bind the event (run after all entry_matrix2 is initialized)
        entry_matrix2[4][3].bind("<FocusOut>", handle_percentage_focusout)  # row 4
        entry_matrix2[5][3].bind("<FocusOut>", handle_percentage_focusout)  # row 5
        entry_matrix2[6][3].bind("<FocusOut>", handle_percentage_focusout)  # row 6
        
        # ---------- Buttons ----------
        btn_row = second_table_row_start + 7 + 1  # Adjust as needed
        
        tk.Button(popup, text="Total", command=enclosure_calculate_total,
                  font=('times new roman', 12, "bold"), bg="#0f4d7d", fg="white")\
                  .grid(row=btn_row, column=1, pady=10, padx=10, sticky="w")
        
        tk.Button(popup, text="Close", command=popup.destroy,
                  font=('times new roman', 12, "bold"), bg="#0f4d7d", fg="white")\
                  .grid(row=btn_row, column=0, pady=10, padx=10, sticky="e")
        
        # Total value variable and display
        enclosure_total_var = tk.StringVar(value="0.00")
        tk.Label(popup, text="Total Volume:", font=('times new roman', 12, "bold"))\
            .grid(row=btn_row, column=2, padx=5, sticky="e")
        
        tk.Entry(popup, textvariable=enclosure_total_var, font=('times new roman', 12),
                 width=15, state='readonly').grid(row=btn_row, column=3, padx=5, sticky="w")

    Button(F3, text="ENCLOSURE FABRICATION:", font=('times new roman', 12, "bold"),
           bg="#0f4d7d", fg="gold", command=lambda: open_enclosure_popup(enclosure_total_var))\
           .grid(row=0, column=0, padx=20, pady=5)

    Label(F3, textvariable=enclosure_total_var, font=("arial", 12,"bold"), bd=3,
          relief=GROOVE, width=20, bg="white",anchor="center",fg="#0f4d7d").grid(row=0, column=1, padx=10, pady=5)

  
    
    wiring_total_var = StringVar(value="₹ 0.00")
    
    def open_wiring_popup(wiring_total_var):
        popup = tk.Toplevel()
        popup.title("Wiring + Accessories")
        popup.geometry("850x460")  # Fits everything without scroll
        popup.configure(bg="#eef2f3")
        popup.resizable(False, False)
    
        headers = ["SIZE", "RATE", "QTY", "TOTAL"]
    
        predefined_items = [
            (35, 350),
            (25, 250),
            (16, 170),
            (10, 110),
            (6, 60),
            (4, 40),
            (2.5, 30),
            (1, 15)
        ]
    
        # Main frame without scrollbars
        frame = tk.Frame(popup, bg="#eef2f3")
        frame.pack(padx=15, pady=15)
    
        # Table Header
        for col, text in enumerate(headers):
            lbl = tk.Label(
                frame,
                text=text,
                font=('Calibri', 12, "bold"),
                bg="#0f4d7d",
                fg="white",
                width=20,
                pady=8,
                bd=1,
                relief="solid"
            )
            lbl.grid(row=0, column=col, padx=1, pady=1)
    
        entry_matrix = []
    
        def wiring_calculate_total():
            total = 0
            for row in entry_matrix:
                try:
                    qty = float(row[2].get())
                    rate = float(row[1].get())
                    row_total = qty * rate
                    row[3].config(state="normal")
                    row[3].delete(0, tk.END)
                    row[3].insert(0, f"{row_total:.2f}")
                    row[3].config(state="readonly")
                    total += row_total
                except:
                    row[3].config(state="normal")
                    row[3].delete(0, tk.END)
                    row[3].insert(0, "")
                    row[3].config(state="readonly")
                    continue
            wiring_total_var.set(f"₹ {total:.2f}")
    
        def wiring_add_row(size="", rate=""):
            row_idx = len(entry_matrix) + 1
            entries = []
            bg_color = "#ffffff" if row_idx % 2 == 0 else "#f1f9ff"
    
            e1 = tk.Entry(frame, width=20, bg=bg_color, font=('Calibri', 11), relief="solid", bd=1)
            e1.insert(0, size)
            e1.grid(row=row_idx, column=0, padx=1, pady=2)
            entries.append(e1)
    
            e2 = tk.Entry(frame, width=20, bg=bg_color, font=('Calibri', 11), relief="solid", bd=1)
            e2.insert(0, rate)
            e2.grid(row=row_idx, column=1, padx=1, pady=2)
            entries.append(e2)
    
            e3 = tk.Entry(frame, width=20, bg=bg_color, font=('Calibri', 11), relief="solid", bd=1)
            e3.grid(row=row_idx, column=2, padx=1, pady=2)
            entries.append(e3)
    
            e4 = tk.Entry(frame, width=20, state="readonly", readonlybackground="#e0f0ff",
                          font=('Calibri', 11), relief="solid", bd=1)
            e4.grid(row=row_idx, column=3, padx=1, pady=2)
            entries.append(e4)
    
            def on_enter(event):
                wiring_calculate_total()
                if entries == entry_matrix[-1]:
                    wiring_add_row()
    
            e3.bind("<Return>", on_enter)
            entry_matrix.append(entries)
    
        for size, rate in predefined_items:
            wiring_add_row(size, rate)
    
        # Bottom Buttons
        btn_frame = tk.Frame(popup, bg="#eef2f3")
        btn_frame.pack(pady=15)
    
        button_style = {
            "font": ('Calibri', 12, "bold"),
            "bg": "#0f4d7d",
            "fg": "white",
            "activebackground": "#155a8a",
            "activeforeground": "white",
            "bd": 0,
            "width": 14,
            "cursor": "hand2"
        }
    
        tk.Button(btn_frame, text="Calculate Total", command=wiring_calculate_total, **button_style)\
            .grid(row=0, column=0, padx=10)
    
        tk.Button(btn_frame, text="Close", command=popup.destroy, **button_style)\
            .grid(row=0, column=1, padx=10)
    
    
    # Main Button to open popup
    Button(F3, text="WIRING + ACCESSORIES", font=('TIMES NEW ROMAN', 12, "bold"),
           bg="#0f4d7d", fg="gold",
           command=lambda: open_wiring_popup(wiring_total_var), padx=12,)\
        .grid(row=1, column=0, padx=20)
    
    # Label to show total
    Label(F3, textvariable=wiring_total_var, font=("ARIAL", 12, "bold"), bd=3,
          relief=GROOVE, width=20, bg="white", anchor="center", fg="#0f4d7d")\
        .grid(row=1, column=1, padx=10)
    

         
    
    
    def open_busbar():
        global table_frame, all_entries, busbar_total_label
    
        popup = tk.Toplevel()
        popup.title("BUS BAR")
        popup.geometry("1400x750")
        popup.configure(bg="white")
    
        canvas_frame = tk.Frame(popup, bg="white")
        canvas_frame.pack(fill=tk.BOTH, expand=True)
    
        canvas = tk.Canvas(canvas_frame, bg="white", highlightthickness=0)
        canvas.grid(row=0, column=0, sticky="nsew")
    
        scrollbar_x = ttk.Scrollbar(canvas_frame, orient="horizontal", command=canvas.xview)
        scrollbar_x.grid(row=1, column=0, sticky="ew")
        canvas.configure(xscrollcommand=scrollbar_x.set)
    
        canvas_frame.grid_rowconfigure(0, weight=1)
        canvas_frame.grid_columnconfigure(0, weight=1)
    
        scrollable_frame = tk.Frame(canvas, bg="white")
        window_id = canvas.create_window((0, 0), window=scrollable_frame, anchor="nw")
    
        def on_frame_configure(event):
            canvas.configure(scrollregion=canvas.bbox("all"))
            canvas.itemconfig(window_id, width=max(scrollable_frame.winfo_reqwidth(), canvas.winfo_width()))
    
        scrollable_frame.bind("<Configure>", on_frame_configure)
    
        def _on_shiftmousewheel(event):
            canvas.xview_scroll(int(-1 * (event.delta / 120)), "units")
    
        canvas.bind_all("<Shift-MouseWheel>", _on_shiftmousewheel)
    
        headers = [
            "L", "W", "RUN", "WEIGHT (KG/M)", "PHASE", "NEUTRAL",
            "PANEL LENGTH", "NO. OF HORI", "TOTAL LENGTH (Mts)",
            "WEIGHT OF THE BUS BAR", "BUS BAR PRICE", "GRAND PRICE"
        ]
        num_cols = len(headers)
        current_row = 0
        all_entries = []
        material_density = tk.DoubleVar(value=0.0)
    
        style = ttk.Style()
        style.theme_use('clam')
        style.configure("CustomCombobox.TCombobox",
                        fieldbackground="white",
                        background="#0f4d7d",
                        foreground="black",
                        font=("Arial", 10),
                        padding=5)
    
        top_frame = tk.Frame(scrollable_frame, bg="#e6f2ff")
        top_frame.grid(row=current_row, column=0, sticky="ew", padx=10, pady=(5, 2))
        tk.Label(top_frame, text="BUS BAR", font=("Arial", 10, "bold"),
                 bg="#e6f2ff", fg="#003366").grid(row=0, column=1)
    
        selected_busbar = tk.StringVar()
        combobox = ttk.Combobox(top_frame, textvariable=selected_busbar,
                                values=[
                                    "VERTICAL-INCOMER FOR B/C", "VERTICAL-O/G FOR Ph1", "VERTICAL-O/G FOR Ph2",
                                    "VERTICAL-APFC O/G", "VERTICAL-INCOMER", "VERTICAL-OUTGOING",
                                    "CONTROL BUS", "EARTH"
                                ],
                                state="readonly", width=35, style="CustomCombobox.TCombobox")
        combobox.set("Select Busbar Type")
        combobox.grid(row=0, column=2, sticky="e", padx=10)
        current_row += 1
    
        sub_frame = tk.Frame(scrollable_frame, bg="#d9fdd3")
        sub_frame.grid(row=current_row, column=0, sticky="ew", padx=10)
    
        def on_material_select(event):
            selection = sub_combobox.get()
            if "Aluminum" in selection:
                material_density.set(0.0027)
            elif "Copper" in selection:
                material_density.set(0.0089)
            else:
                material_density.set(0.0)
    
        sub_combobox = ttk.Combobox(sub_frame, state="readonly",
                                     values=["Aluminum = 0.0027", "Copper = 0.0089"],
                                     width=20, font=("Arial", 10),
                                     style="CustomCombobox.TCombobox")
        sub_combobox.set(" ")
        sub_combobox.bind("<<ComboboxSelected>>", on_material_select)
        sub_combobox.grid(row=0, column=0, sticky="w", padx=10)
    
        tk.Label(sub_frame, text="HORIZONTAL - MAIN TRFO & DG1",
                 font=("Arial", 10, "bold"), bg="#d9fdd3", fg="#004d00") \
            .grid(row=0, column=1, sticky="w", padx=10)
        current_row += 1
    
        busbar_data = {
            "VERTICAL-INCOMER FOR B/C": [["25", "10", "1", "", "3", "", "1600", "1", "", "", "400", ""]],
            "VERTICAL-O/G FOR Ph1": [["30", "5", "2", "", "1", "1", "1800", "2", "", "", "500", ""]],
            "VERTICAL-O/G FOR Ph2": [["40", "5", "2", "", "2", "1", "1500", "1", "", "", "520", ""]],
            "VERTICAL-APFC O/G": [["50", "10", "2", "", "1", "1", "1800", "1", "", "", "600", ""]],
            "VERTICAL-INCOMER": [["20", "10", "1", "", "1", "0", "1600", "1", "", "", "450", ""]],
            "VERTICAL-OUTGOING": [["20", "10", "1", "", "0", "1", "1600", "1", "", "", "450", ""]],
            "CONTROL BUS": [["10", "5", "1", "", "1", "1", "1600", "2", "", "", "300", ""]],
            "EARTH": [["16", "5", "1", "", "0", "1", "1600", "1", "", "", "200", ""]]
        }
    
        def on_last_cell_tab(event):
            widget = event.widget
            if all_entries and widget == all_entries[-1][-1]:
                add_new_row()
                return "break"
    
        def add_new_row():
            row_index = len(all_entries) + 1
            row_entries = []
            for c in range(num_cols):
                e = tk.Entry(table_frame, font=("Arial", 10), relief="solid", justify="center")
                e.grid(row=row_index, column=c, padx=1, pady=1, sticky="nsew")
                if c == num_cols - 1:
                    e.bind("<Tab>", on_last_cell_tab)
                row_entries.append(e)
            all_entries.append(row_entries)
    
        def create_section(title, data):
            nonlocal current_row
            global table_frame
    
            section_title = tk.Label(scrollable_frame, text=title, font=("Arial", 10, "bold"),
                                     bg="#f2f2f2", anchor="w")
            section_title.grid(row=current_row, column=0, sticky="ew", padx=10, pady=(10, 2))
            current_row += 1
    
            table_frame = tk.Frame(scrollable_frame, bg="white")
            table_frame.grid(row=current_row, column=0, sticky="ew", padx=10)
    
            for i in range(num_cols):
                table_frame.grid_columnconfigure(i, weight=1)
    
            for col, header in enumerate(headers):
                lbl = tk.Label(table_frame, text=header, font=("Arial", 9, "bold"),
                               bg="#ffe0b3", relief="solid", bd=1, width=14, anchor="center")
                lbl.grid(row=0, column=col, sticky="nsew", padx=1, pady=1)
    
            for r_idx, row_data in enumerate(data, start=1):
                row_entries = []
                for c in range(num_cols):
                    e = tk.Entry(table_frame, font=("Arial", 10), relief="solid", justify="center")
                    e.grid(row=r_idx, column=c, padx=1, pady=1, sticky="nsew")
                    if c < len(row_data):
                        e.insert(0, row_data[c])
                    if c == num_cols - 1:
                        e.bind("<Tab>", on_last_cell_tab)
                    row_entries.append(e)
                all_entries.append(row_entries)
    
            current_row += 1
    
        def calc_all():
            density = material_density.get()
            for row in all_entries:
                try:
                    l = float(row[0].get() or 0)
                    w = float(row[1].get() or 0)
                    run = float(row[2].get() or 0)
                    weight_kgm = l * w * run * density
                    row[3].delete(0, tk.END)
                    row[3].insert(0, f"{weight_kgm:.4f}")
    
                    phase = int(row[4].get() or 0)
                    neutral = int(row[5].get() or 0)
                    panel_len = float(row[6].get() or 0)
                    no_of_hori = float(row[7].get() or 0)
                    total_len = (panel_len / 1000) * no_of_hori * (phase + neutral)
                    row[8].delete(0, tk.END)
                    row[8].insert(0, f"{total_len:.2f}")
    
                    weight_busbar = total_len * weight_kgm
                    row[9].delete(0, tk.END)
                    row[9].insert(0, f"{weight_busbar:.2f}")
    
                    price = float(row[10].get() or 0)
                    grand = weight_busbar * price
                    row[11].delete(0, tk.END)
                    row[11].insert(0, f"{grand:,.2f}")
                except Exception as e:
                    print("Row Error:", e)
    
        def show_grand_total():
            total = 0.0
            for row in all_entries:
                try:
                    val = float(row[11].get().replace(",", "") or 0)
                    total += val
                except:
                    pass
            busbar_total_label.config(text=f"{total:,.2f}")
    
        def on_combobox_select(event):
            selection = selected_busbar.get()
            if selection in busbar_data:
                create_section(selection, busbar_data[selection])
    
        combobox.bind("<<ComboboxSelected>>", on_combobox_select)
    
        bottom_button_frame = tk.Frame(popup, bg="white")
        bottom_button_frame.pack(fill=tk.X, side=tk.BOTTOM, anchor="e")
    
        calculate_btn = tk.Button(bottom_button_frame, text="Calculate All", command=calc_all,
                                   bg="#007bff", fg="white", font=("Arial", 10, "bold"))
        calculate_btn.pack(side=tk.RIGHT, padx=10, pady=5)
    
        total_btn = tk.Button(bottom_button_frame, text="Total", command=show_grand_total,
                              bg="#28a745", fg="white", font=("Arial", 10, "bold"))
        total_btn.pack(side=tk.RIGHT, padx=10, pady=5)
     
    # Button and label in F3
    Button(F3, text="BUS BAR:", font=('times new roman', 12, "bold"),
           bg="#0f4d7d", fg="gold", command=open_busbar)\
        .grid(row=1, column=2, padx=20, pady=5)
    
    global busbar_total_label
    busbar_total_label = Label(F3, font=("arial", 12,"bold"), bd=3, relief=GROOVE, width=20,bg="white",anchor="center",fg="#0f4d7d")
    busbar_total_label.grid(row=1, column=3, padx=10, pady=5)
                

    Label(F3, text="LBOUR:", font=('times new roman', 12, "bold"), bg="#0f4d7d", fg="gold")\
        .grid(row=2, column=0, padx=20)
    Entry(F3, font=("arial", 12), bd=3, relief=GROOVE).grid(row=2, column=1, padx=10)

    Label(F3, text="TRANSPORTATION:", font=('times new roman', 15, "bold"), bg="#0f4d7d", fg="gold")\
        .grid(row=0, column=2)
    Entry(F3, font=("arial", 15), bd=3, relief=GROOVE).grid(row=0, column=3, padx=10)


    Button(F3, text="GENERATE BILL", font=('arial', 12, 'bold'), bg="white", fg="black", pady=5, padx=5)\
        .grid(row=2, column=6, pady=5, padx=5)

    Label(F3, font=('times new roman', 15, "bold"), bg="#0f4d7d", fg="white")\
            .grid(row=2, column=8, padx=20, pady=5)

    lp_total_var = tk.StringVar(value="0.00")
    Label(F3, text="LP. GROSS PRICE", font=("arial", 12, "bold"), bg="#0f4d7d", fg="gold").grid(row=0, column=7, padx=10, pady=5, sticky="w")
    Entry(F3, textvariable=lp_total_var, font=("arial", 12), bd=3, state="readonly").grid(row=0, column=8, padx=10, pady=5)

    net_disc_price_var = StringVar(value="₹ 0.00")
    Label(F3, text="NET DISC. PRICE:", font=('times new roman', 12, "bold"), bg="#0f4d7d", fg="gold")\
    .grid(row=1, column=7, padx=20, pady=5)
    Entry(F3, font=("arial", 12), bd=3, relief=GROOVE, textvariable=net_disc_price_var, state="readonly")\
    .grid(row=1, column=8, padx=10, pady=5)
    
   
    add_entry_row()
    root.mainloop()

Bill_entry()

###### Deveshri Code ######

In [ ]:
w                                             # Connect to SQL Server
conn = odbc.connect(
    "DRIVER={ODBC Driver 17 for SQL Server};"
    "SERVER=DESKTOP-7I9UC6G\\SQLEXPRESS;"
    "DATABASE=python;"
    "Trusted_Connection=yes;"
)
cursor = conn.cursor()

def make_frame_clickable(frame, title, callback):
    frame.bind("<Button-1>", lambda e: callback(title))
    for child in frame.winfo_children():
        child.bind("<Button-1>", lambda e: callback(title))

root=Tk()
root.title('Dashboard')
root.geometry('1350x690+0+0')  # +0+0 means distance of frame will be 0 from x and y axis
root.config(bg='#5761B2')

from tkinter import Toplevel

notifications = []

notif_frame = ttk.Frame(root)
notif_frame.pack(pady=20)

# Function to fetch recent changes from database
def fetch_notifications():
    cursor.execute("SELECT TOP 5 'New sale recorded!' FROM Sales ORDER BY SaleDate DESC")
    new_sales = cursor.fetchall()
    
    cursor.execute("SELECT TOP 5 'Inventory updated!' FROM Inventory ORDER BY LastUpdated DESC")
    new_inventory = cursor.fetchall()
    
    cursor.execute("SELECT TOP 5 'Supplier added successfully!' FROM Suppliers ORDER BY ID DESC")
    new_suppliers = cursor.fetchall()

    latest_notifications = [notif[0] for notif in (new_sales + new_inventory + new_suppliers)]
    
    return latest_notifications

def update_notification_display(notif_window, scroll_frame):
    for widget in scroll_frame.winfo_children():
        widget.destroy()  # Clear old notifications

    Label(scroll_frame, text="Recent Notifications:", font=('times new roman', 15, 'bold')).pack(pady=10)

    notifications = fetch_notifications()
    
    for notif in notifications:
        Label(scroll_frame, text=f"- {notif}", font=('times new roman', 13), fg="white", bg="#2C3E50", padx=10, pady=5).pack(anchor="w", padx=20)

    # Re-run this function every 5 seconds
    notif_window.after(5000, lambda: update_notification_display(notif_window, scroll_frame))

# Function to open notifications window
def open_notifications():
    notif_window = Toplevel(root)
    notif_window.title("Notifications")
    notif_window.geometry("400x300+500+200")
    notif_window.config(bg="#2C3E50") 
    
    Label(notif_window, text="🔔 Notifications:", font=('times new roman', 20, 'bold')).pack(pady=10)

    canvas = Canvas(notif_window, bg="#2C3E50")
    scrollbar = Scrollbar(notif_window, orient="vertical", command=canvas.yview)
    scroll_frame = Frame(canvas, bg="#2C3E50")

    scroll_frame.bind("<Configure>", lambda e: canvas.configure(scrollregion=canvas.bbox("all")))

    canvas.create_window((0, 0), window=scroll_frame, anchor="nw")
    canvas.configure(yscrollcommand=scrollbar.set)

    update_notification_display(notif_window, scroll_frame)
   
    canvas.pack(side="left", fill="both", expand=True)
    scrollbar.pack(side="right", fill="y")

    Button(notif_window, text="Close", command=notif_window.destroy).pack(pady=15)
                                                                                                                                  
bg_image=PhotoImage(file='inventory (2).png')
titleLabel=Label(root, image=bg_image ,compound=LEFT,  text='Inventory Managment System', font=('times new roman', 40, 'bold'), bg='#010c48', fg='white', anchor='w', padx=20)
titleLabel.place(x=0,y=0, relwidth=1)

bell_icon = PhotoImage(file="bell.png")  # Ensure you have a bell.png file
bell_button = Button(root, image=bell_icon, command=open_notifications, bd=2)
bell_button.place(x=1120, y=17)  # Adjust position as needed970, y=150

LogoutButton=Button(root, text='Logout', font=('times new roman', 20, 'bold'))
LogoutButton.place(x=1200,y=9)

subtitle=Label(root, text='Welcome Admin\t\t Date: 08-07-2024\t\t Time: 12:36:13 pm', font=('times new roman', 15),bg='#4d636d', fg='white')
subtitle.place(x=0, y=68, relwidth=1)

leftFrame=Frame(root)
leftFrame.place(x=0, y=96, width=200, height=595)

logoImage=PhotoImage(file='checklist (1).png')
ImageLabel=Label(leftFrame, image=logoImage)
ImageLabel.pack()

menuLabel=Label(leftFrame, text='Menu', font=('times new roman',20),bg='#009688')
menuLabel.pack(fill=X)

emp_image=PhotoImage(file='employee.png')
employee_button=Button(leftFrame, image=emp_image ,compound=LEFT,  text='Employees', font=('times new roman', 20, 'bold'))
employee_button.pack(fill=X)

product_image=PhotoImage(file='box.png')
Product_button=Button(leftFrame, image=product_image, compound=LEFT, text='Products', font=('times new roman', 20, 'bold'))
Product_button.pack(fill=X)

panel_image=PhotoImage(file='solar.png')
Panel_button=Button(leftFrame, image=panel_image ,compound=LEFT,  text='Panels', font=('times new roman', 20, 'bold'))
Panel_button.pack(fill=X)

Inventory_image=PhotoImage(file='inventory (1).png')
Inventory_button=Button(leftFrame, image=Inventory_image ,compound=LEFT, text='Inventory', font=('times new roman', 20, 'bold'))
Inventory_button.pack(fill=X)

billing_image=PhotoImage(file='bill.png')
Billing_button=Button(leftFrame, image=billing_image ,compound=LEFT, text='Billing', font=('times new roman', 20, 'bold'))
Billing_button.pack(fill=X)

sales_image=PhotoImage(file='increase.png')
sales_button=Button(leftFrame, image=sales_image ,compound=LEFT, text='Sales', font=('times new roman', 20, 'bold'))
sales_button.pack(fill=X)

help_image=PhotoImage(file='chat.png')
help_button=Button(leftFrame, image=help_image ,compound=LEFT, text='Help', font=('times new roman', 20, 'bold'))
help_button.pack(fill=X)

exit_image=PhotoImage(file='exit.png')
exit_button=Button(leftFrame, image=exit_image ,compound=LEFT, text='Exit', font=('times new roman', 20, 'bold'))
exit_button.pack(fill=X)


graphtitle=Label(root, text='📊 Buisness Overview', font=('times new roman', 17, 'bold'), bg='#7BD5F5')
graphtitle.place(x=970, y=130)

fig, ax = plt.subplots(figsize=(5.5, 5.1))
canvas = FigureCanvasTkAgg(fig, master=root)
canvas.get_tk_widget().place(x=780, y=170)

def update_graph(data, title, chart_type="bar"):
    ax.clear()
    ax.set_title(title, fontsize=14)

    if not data:
        ax.text(0.5, 0.5, "No data to display", fontsize=12, ha='center', va='center')
        canvas.draw()
        canvas.get_tk_widget().update_idletasks()
        return

    labels = [str(row[0]) for row in data]

    def safe_float(val):
        try:
            return float(val)
        except (ValueError, TypeError):
            return 0
            
    values = [safe_float(row[1]) for row in data]

    # Plot based on chart type
    if chart_type == "bar":
        ax.bar(labels, values, color="#61AFEF")
    elif chart_type == "line":
        ax.plot(labels, values, color="#EF476F", marker='o', linestyle='-')
    elif chart_type == "pie":
        if sum(values) == 0:
            ax.text(0.5, 0.5, "No valid data for pie chart", fontsize=12, ha='center', va='center')
        else:
            ax.pie(values, labels=labels, autopct='%1.1f%%', startangle=140)
            ax.axis('equal')  # Make pie chart circular
    else:
        ax.text(0.5, 0.5, "Invalid chart type", fontsize=12, ha='center', va='center')

    # Styling for bar and line charts
    if chart_type in ("bar", "line"):
        # Rotate x-axis labels
        plt.setp(ax.get_xticklabels(), rotation=45, ha='right', va='top')
        # Padding
        ax.tick_params(axis='x', pad=10)

    # Adjust layout to prevent clipping
    fig.tight_layout()

    # Redraw the canvas
    canvas.draw()
    canvas.get_tk_widget().update_idletasks()


def fetch_employee_data():
    cursor.execute("""
        SELECT 
            e.ID, e.Name, e.Designation, e.BasicSalary, e.Status,
            FORMAT(e.JoiningDate, 'yyyy-MM-dd'), FORMAT(e.ResigningDate, 'yyyy-MM-dd'),
            s.OTDayAmount, s.OTHours, s.AdvanceTaken, s.AdvanceDeducted, s.TotalSalary
        FROM Employees1 e
        LEFT JOIN EmployeeStats s ON e.ID = s.EmployeeID
    """)
    return cursor.fetchall()

def show_data_in_window(title, data, columns, show_buttons=False, show_filters=False, show_supplier_search=False):
    data_window = Toplevel(root)
    data_window.title(title)
    data_window.geometry(f"{root.winfo_screenwidth()}x{root.winfo_screenheight()}")
    data_window.state('zoomed')
    data_window.config(bg="#96C3EB")

    Label(data_window, text=title, font=('times new roman', 30, 'bold'),
          bg="#158FAD", fg="white", pady=20).pack()

    # Filters (only for employee)
    # Filters for employee
    if show_filters:
        search_frame = Frame(data_window, bg="#96C3EB")
        search_frame.pack(pady=10)

        search_var = StringVar()
        status_var = StringVar(value="All")

        Entry(search_frame, textvariable=search_var, font=('times new roman', 14), width=30).pack(side=LEFT, padx=5)
        Button(search_frame, text="Search", font=('times new roman', 12, 'bold'),
               command=lambda: search_tree(tree, search_var.get(), status_var.get())).pack(side=LEFT, padx=5)

        Label(search_frame, text="Filter by Status:", font=('times new roman', 12, 'bold'),
              bg="#96C3EB").pack(side=LEFT, padx=10)

        filter_menu = ttk.Combobox(search_frame, textvariable=status_var,
                                   values=["All", "Working", "Resigned"], state="readonly")
        filter_menu.pack(side=LEFT)

    # Supplier Search
    elif show_supplier_search:
        search_frame = Frame(data_window, bg="#96C3EB")
        search_frame.pack(pady=10)

        search_var = StringVar()
        Entry(search_frame, textvariable=search_var, font=('times new roman', 14), width=30).pack(side=LEFT, padx=5)
        Button(search_frame, text="Search", font=('times new roman', 12, 'bold'),
               command=lambda: search_supplier_tree(tree, search_var.get())).pack(side=LEFT, padx=5)


    # Tree setup
    frame = Frame(data_window, bg="#96C3EB", bd=5, relief=SOLID)
    frame.pack(expand=True, fill="both", padx=20, pady=20)

    v_scrollbar = Scrollbar(frame, orient=VERTICAL)
    h_scrollbar = Scrollbar(frame, orient=HORIZONTAL)
    v_scrollbar.pack(side=RIGHT, fill=Y)
    h_scrollbar.pack(side=BOTTOM, fill=X)

    tree = ttk.Treeview(frame, columns=columns, show="headings",
                        yscrollcommand=v_scrollbar.set, xscrollcommand=h_scrollbar.set)
    tree.pack(expand=True, fill="both")
    v_scrollbar.config(command=tree.yview)
    h_scrollbar.config(command=tree.xview)

    for col in columns:
        tree.heading(col, text=col, anchor=CENTER)
        tree.column(col, anchor=CENTER, width=200)

    style = ttk.Style()
    style.configure("Treeview", font=("times new roman", 15), rowheight=30)
    style.configure("Treeview.Heading", font=("times new roman", 18, "bold"))

    for item in data:
        tree.insert("", "end", values=[str(value) for value in item])

    # Add buttons if required
    if show_buttons:
        btn_frame = Frame(data_window, bg="#96C3EB")
        btn_frame.pack(pady=10)

        if title == "Staffing Overview":
            Button(btn_frame, text="Add", font=('times new roman', 15, 'bold'),
                   bg="green", fg="white", width=12, command=open_add_employee_form).pack(side=LEFT, padx=15)
            Button(btn_frame, text="Edit", font=('times new roman', 15, 'bold'),
                   bg="red", fg="white", width=12, command=lambda: open_edit_employee_form(tree)).pack(side=LEFT, padx=15)
        elif title == "Supplier Hub":
            Button(btn_frame, text="Add", font=('times new roman', 15, 'bold'),
                   bg="green", fg="white", width=12, command=open_add_supplier_form).pack(side=LEFT, padx=15)
            Button(btn_frame, text="Edit", font=('times new roman', 15, 'bold'),
                   bg="red", fg="white", width=12, command=lambda: open_edit_supplier_form(tree)).pack(side=LEFT, padx=15)

    def on_double_click(event):
        region = tree.identify("region", event.x, event.y)
        if region != "cell":
            return

        row_id = tree.identify_row(event.y)
        column_id = tree.identify_column(event.x)

        x, y, width, height = tree.bbox(row_id, column_id)
        item = tree.item(row_id)
        column_index = int(column_id[1:]) - 1
        old_value = item['values'][column_index]

        entry = Entry(tree, font=('times new roman', 14))
        entry.place(x=x, y=y, width=width, height=height)
        entry.insert(0, old_value)
        entry.focus()

        def save_edit(event):
            new_value = entry.get()
            entry.destroy()
            tree.set(row_id, column=columns[column_index], value=new_value)

            emp_id = item['values'][0]  # assuming first column is ID
            field = columns[column_index]

            # Only update editable fields
            editable_fields = ["Name", "Designation", "BasicSalary", "Status"]
            if field in editable_fields:
                try:
                    cursor.execute(f"UPDATE Employees1 SET {field} = ? WHERE ID = ?", (new_value, emp_id))
                    conn.commit()
                except Exception as e:
                    messagebox.showerror("Error", f"Failed to update database: {e}")

        entry.bind("<Return>", save_edit)
        entry.bind("<FocusOut>", lambda e: entry.destroy())

    if title == "Staffing Overview":
        tree.bind("<Double-1>", on_double_click)

    Button(data_window, text="Close", font=('times new roman', 15, 'bold'),
           command=data_window.destroy, width=12).pack(pady=10)
 
def search_tree(tree, keyword, status_filter):
    keyword = keyword.lower()
    for row in tree.get_children():
        tree.delete(row)

    query = """
        SELECT 
            e.ID, e.Name, e.Designation, e.BasicSalary, e.Status,
            FORMAT(e.JoiningDate, 'yyyy-MM-dd'), FORMAT(e.ResigningDate, 'yyyy-MM-dd'),
            s.OTDayAmount, s.OTHours, s.AdvanceTaken, s.AdvanceDeducted, s.TotalSalary
        FROM Employees1 e
        LEFT JOIN EmployeeStats s ON e.ID = s.EmployeeID
        WHERE (LOWER(e.Name) LIKE ? OR LOWER(e.Designation) LIKE ?)
    """

    params = [f"%{keyword}%"] * 2
    if status_filter != "All":
        query += " AND e.Status = ?"
        params.append(status_filter)

    cursor.execute(query, params)
    results = cursor.fetchall()
    for item in results:
        tree.insert("", "end", values=item)

def search_supplier_tree(tree, keyword):
    keyword = keyword.lower()
    for row in tree.get_children():
        tree.delete(row)

    query = """
        SELECT ID, Name, Company, Contact, Materials 
        FROM Suppliers
        WHERE LOWER(Name) LIKE ? OR LOWER(Company) LIKE ? OR LOWER(Materials) LIKE ?
    """
    params = [f"%{keyword}%"] * 3
    cursor.execute(query, params)
    results = cursor.fetchall()
    for item in results:
        tree.insert("", "end", values=item)


def open_employee_window(title):
    employees = fetch_employee_data()
    columns = ["ID", "Name", "Designation", "BasicSalary", "Status",
               "JoiningDate", "ResigningDate", "OTDayAmount", "OTHours", "AdvanceTaken", "AdvanceDeducted", "TotalSalary"]
    show_data_in_window(title, employees, columns)

def open_add_employee_form():
    width, height = 600, 500
    screen_width = root.winfo_screenwidth()
    screen_height = root.winfo_screenheight()
    x = (screen_width // 2) - (width // 2)
    y = (screen_height // 2) - (height // 2)
    
    form = Toplevel(root)
    form.title("Add Employee")
    form.geometry(f"{width}x{height}+{x}+{y}")
    form.config(bg="#F2F3F4")  # Light grey background
    form.resizable(False, False)

    fields = ["Name", "Designation", "BasicSalary", "Status"]
    entry_vars = {field: StringVar() for field in fields}

    header = Label(form, text="Add New Employee", font=("times new roman", 20, 'bold'),
                   bg="#1ABC9C", fg="white", pady=10)
    header.pack(fill=X)

    container = Frame(form, bg="#F2F3F4")
    container.pack(pady=20)

    for idx, field in enumerate(fields):
        Label(container, text=field + ":", font=("times new roman", 13), bg="#F2F3F4")\
            .grid(row=idx+1, column=0, pady=8, padx=10, sticky=W)
        Entry(container, textvariable=entry_vars[field], font=("times new roman", 12), width=28)\
            .grid(row=idx+1, column=1, pady=8, padx=10)

    joining_row = len(fields)+1

    Label(container, text="Joining Date:", font=("times new roman", 13), bg="#F2F3F4")\
        .grid(row=joining_row, column=0, pady=8, padx=10, sticky=W)
    joining_date = DateEntry(container, width=20, font=("times new roman", 12))
    joining_date.grid(row=joining_row, column=1, pady=8, padx=10)

    Label(container, text="Resigning Date:", font=("times new roman", 13), bg="#F2F3F4")\
        .grid(row=joining_row+1, column=0, pady=8, padx=10, sticky=W)
    resigning_date = DateEntry(container, width=20, font=("times new roman", 12))
    resigning_date.grid(row=joining_row+1, column=1, pady=8, padx=10)
    resigning_date.config(state="disabled")

    resign_check_var = BooleanVar()

    def toggle_resign_entry():
        resigning_date.config(state="normal" if resign_check_var.get() else "disabled")

    Checkbutton(container, text="Include Resigning Date", variable=resign_check_var,
                command=toggle_resign_entry, bg="#F0F8FF").grid(row=joining_row+2, columnspan=2, pady=8)

    def submit():
        try:
            name = entry_vars["Name"].get().strip()
            designation = entry_vars["Designation"].get().strip()
            basic_salary = float(entry_vars["BasicSalary"].get())
            status = entry_vars["Status"].get().strip()
            join_date_value = joining_date.get_date()
            resign_date_value = resigning_date.get_date() if resign_check_var.get() else None

            if not name or not designation or basic_salary <= 0:
                messagebox.showerror("Input Error", "Please fill all fields correctly.")
                return

            if resign_date_value:
                status = "Resigned"

            cursor.execute("""
                INSERT INTO Employees1 (Name, Designation, BasicSalary, Status, JoiningDate, ResigningDate)
                VALUES (?, ?, ?, ?, ?, ?)
            """, (
                 name, designation, basic_salary,
                status, join_date_value, resign_date_value
            ))
            conn.commit()

            messagebox.showinfo("Success", "Employee added successfully!")
            form.destroy()

        except Exception as e:
            messagebox.showerror("Error", f"Could not add employee:\n{str(e)}")

    Button(form, text="Submit", font=("times new roman", 14, 'bold'),
           bg="#2980B9", fg="white", width=12, command=submit).pack(pady=20)


def open_edit_employee_form(tree):
    selected = tree.focus()
    if not selected:
        messagebox.showerror("Error", "Please select an employee to edit.")
        return

    values = tree.item(selected)['values']

    # Centered window size
    width, height = 600, 500
    screen_width = root.winfo_screenwidth()
    screen_height = root.winfo_screenheight()
    x = (screen_width // 2) - (width // 2)
    y = (screen_height // 2) - (height // 2)

    form = Toplevel(root)
    form.title("Edit Employee")
    form.geometry(f"{width}x{height}+{x}+{y}")
    form.config(bg="#F2F3F4")  # Light grey background
    form.resizable(False, False)

    # Header
    Label(form, text="Edit Employee Details", font=("Helvetica", 20, 'bold'),
          bg="#5DADE2", fg="white", pady=12).pack(fill=X)

    # Container
    container = Frame(form, bg="#F2F3F4")
    container.pack(pady=20, padx=30)

    fields = ["Name", "Designation", "BasicSalary", "Status", "JoiningDate", "ResigningDate"]
    entry_vars = {field: StringVar(value=str(values[i+1])) for i, field in enumerate(fields)}

    for idx, field in enumerate(fields):
        Label(container, text=field + ":", font=("Helvetica", 13, 'bold'), bg="#F2F3F4", anchor=W).grid(
            row=idx, column=0, sticky=W, pady=8, padx=5)
        
        if field in ["JoiningDate", "ResigningDate"]:
            entry = DateEntry(container, textvariable=entry_vars[field], width=20, font=("Helvetica", 12))
        else:
            entry = Entry(container, textvariable=entry_vars[field], font=("Helvetica", 12), width=30)
        
        entry.grid(row=idx, column=1, pady=8, padx=10)

    def update():
        try:
            emp_id = values[0]
            status = entry_vars["Status"].get()
            if entry_vars["ResigningDate"].get():
                status = "Resigned"

            cursor.execute("""
                UPDATE Employees1 SET Name=?, Designation=?, BasicSalary=?, Status=?, JoiningDate=?, ResigningDate=?
                WHERE ID=?
            """, (
                entry_vars["Name"].get(), entry_vars["Designation"].get(), float(entry_vars["BasicSalary"].get()),
                status, entry_vars["JoiningDate"].get(), entry_vars["ResigningDate"].get(), emp_id
            ))
            conn.commit()
            messagebox.showinfo("Success", "Employee updated successfully!")
            form.destroy()
        except Exception as e:
            messagebox.showerror("Error", f"Could not update employee:\n{str(e)}")

    # Update button
    Button(form, text="Update", font=("Helvetica", 14, 'bold'),
           bg="#27AE60", fg="white", activebackground="#1E8449", width=14,
           command=update).pack(pady=20)

def open_add_supplier_form():
    width, height = 500, 400
    x = (root.winfo_screenwidth() // 2) - (width // 2)
    y = (root.winfo_screenheight() // 2) - (height // 2)

    form = Toplevel(root)
    form.title("Add Supplier")
    form.geometry(f"{width}x{height}+{x}+{y}")
    form.config(bg="#F2F3F4")
    form.resizable(False, False)

    fields = ["Name", "Company", "Contact", "Materials"]
    entry_vars = {field: StringVar() for field in fields}

    Label(form, text="Add New Supplier", font=("Helvetica", 18, 'bold'),
          bg="#5DADE2", fg="white", pady=10).pack(fill=X)

    container = Frame(form, bg="#F2F3F4")
    container.pack(pady=20, padx=30)

    for idx, field in enumerate(fields):
        Label(container, text=field + ":", font=("Helvetica", 13, 'bold'), bg="#F2F3F4").grid(
            row=idx, column=0, sticky=W, pady=8, padx=5)
        Entry(container, textvariable=entry_vars[field], font=("Helvetica", 12), width=30).grid(
            row=idx, column=1, pady=8, padx=5)

    def submit_supplier():
        try:
            cursor.execute("""
                INSERT INTO Suppliers (Name, Company, Contact, Materials)
                VALUES (?, ?, ?, ?)
            """, (
                entry_vars["Name"].get(), entry_vars["Company"].get(),
                entry_vars["Contact"].get(), entry_vars["Materials"].get()
            ))
            conn.commit()
            messagebox.showinfo("Success", "Supplier added successfully!")
            form.destroy()
        except Exception as e:
            messagebox.showerror("Error", f"Could not add supplier:\n{str(e)}")

    Button(form, text="Submit", font=("Helvetica", 14, 'bold'),
           bg="#27AE60", fg="white", width=14, command=submit_supplier).pack(pady=20)

def open_edit_supplier_form(tree):
    selected = tree.focus()
    if not selected:
        messagebox.showerror("Error", "Please select a supplier to edit.")
        return

    values = tree.item(selected)['values']

    width, height = 500, 400
    x = (root.winfo_screenwidth() // 2) - (width // 2)
    y = (root.winfo_screenheight() // 2) - (height // 2)

    form = Toplevel(root)
    form.title("Edit Supplier")
    form.geometry(f"{width}x{height}+{x}+{y}")
    form.config(bg="#F2F3F4")
    form.resizable(False, False)

    fields = ["Name", "Company", "Contact", "Materials"]
    entry_vars = {field: StringVar(value=str(values[i+1])) for i, field in enumerate(fields)}

    Label(form, text="Edit Supplier Details", font=("Helvetica", 18, 'bold'),
          bg="#5DADE2", fg="white", pady=10).pack(fill=X)

    container = Frame(form, bg="#F2F3F4")
    container.pack(pady=20, padx=30)

    for idx, field in enumerate(fields):
        Label(container, text=field + ":", font=("Helvetica", 13, 'bold'), bg="#F2F3F4").grid(
            row=idx, column=0, sticky=W, pady=8, padx=5)
        Entry(container, textvariable=entry_vars[field], font=("Helvetica", 12), width=30).grid(
            row=idx, column=1, pady=8, padx=5)

    def update_supplier():
        try:
            cursor.execute("""
                UPDATE Suppliers SET Name=?, Company=?, Contact=?, Materials=?
                WHERE ID=?
            """, (
                entry_vars["Name"].get(), entry_vars["Company"].get(),
                entry_vars["Contact"].get(), entry_vars["Materials"].get(),
                values[0]
            ))
            conn.commit()
            messagebox.showinfo("Success", "Supplier updated successfully!")
            form.destroy()
        except Exception as e:
            messagebox.showerror("Error", f"Could not update supplier:\n{str(e)}")

    Button(form, text="Update", font=("Helvetica", 14, 'bold'),
           bg="#27AE60", fg="white", width=14, command=update_supplier).pack(pady=20)


def open_employee_window(title):
    def refresh():
        open_employee_window(title)
    employees = fetch_employee_data()
    columns = ["ID", "Name", "Designation", "BasicSalary", "Status",
               "JoiningDate", "ResigningDate", "OTDayAmount", "OTHours", "AdvanceTaken", "AdvanceDeducted", "TotalSalary"]
    show_data_in_window(title, employees, columns, show_buttons=True, show_filters=True, show_supplier_search=False)

emp_frame=Frame(root, bg='#2C3E50', bd=3,  relief=RIDGE)
emp_frame.place(x=230, y=125, height=170, width=230)
total_emp_image=PhotoImage(file='team.png')
total_emp_image_label=Label(emp_frame, image=total_emp_image, bg='#2C3E50')
total_emp_image_label.pack()
total_emp_label=Label(emp_frame, text='Staffing Overview', font=('times new roman', 15, 'bold') ,bg='#2C3E50', fg='white')
total_emp_label.pack()
total_emp_count_label=Label(emp_frame, text='0', bg='#2C3E50',fg='white', font=('times new roman', 20, 'bold'))
total_emp_count_label.pack()
make_frame_clickable(emp_frame, 'Staffing Overview', open_employee_window)

def fetch_supplier_data():
    cursor.execute("SELECT ID, Name, Company, Contact, Materials FROM Suppliers")
    suppliers = cursor.fetchall()
    return suppliers
    
def open_supplier_window(title):
    from collections import Counter 
    suppliers = fetch_supplier_data()
    show_data_in_window(title, suppliers, ["ID", "Name", "Company", "Contact", "Materials"],
                        show_buttons=True, show_filters=False, show_supplier_search=True)
    
sup_frame=Frame(root, bg='#2C3E50', bd=3, relief=RIDGE)
sup_frame.place(x=520, y=125, height=170, width=230)
total_sup_image=PhotoImage(file='supplier.png')
total_sup_image_label=Label(sup_frame, image=total_sup_image, bg='#2C3E50')
total_sup_image_label.pack()
total_sup_label=Label(sup_frame, text='Supplier Hub', font=('times new roman', 15, 'bold') ,bg='#2C3E50', fg='white')
total_sup_label.pack()
total_sup_count_label=Label(sup_frame, text='0', bg='#2C3E50',fg='white', font=('times new roman', 20, 'bold'))
total_sup_count_label.pack()
make_frame_clickable(sup_frame, 'Supplier Hub', open_supplier_window)

def open_revenue_window(title):
    revenue_data = [("Q1", 20000), ("Q2", 30000), ("Q3", 40000), ("Q4", 50000)]
    update_graph(revenue_data, "Quarterly Revenue", chart_type="line")
    messagebox.showinfo("Coming Soon", "Revenue insights coming soon!")

revenue_frame=Frame(root, bg='#2C3E50', bd=3, relief=RIDGE)
revenue_frame.place(x=230, y=310, height=170, width=230)
total_revenue_image=PhotoImage(file='revenue.png')
total_revenue_image_label=Label(revenue_frame, image=total_revenue_image, bg='#2C3E50')
total_revenue_image_label.pack()
total_revenue_label=Label(revenue_frame, text='Revenue Insights', font=('times new roman', 15, 'bold') ,bg='#2C3E50', fg='white')
total_revenue_label.pack()
total_revenue_count_label=Label(revenue_frame, text='0', bg='#2C3E50',fg='white', font=('times new roman', 20, 'bold'))
total_revenue_count_label.pack()
make_frame_clickable(revenue_frame, 'Revenue Insights', open_revenue_window)


def show_data_in_product_window(title, data, columns):
    data_window = Toplevel(root)
    data_window.title(title)
    data_window.geometry(f"{root.winfo_screenwidth()}x{root.winfo_screenheight()}")
    data_window.state('zoomed')
    data_window.config(bg="#96C3EB")

    Label(data_window, text=title, font=('times new roman', 30, 'bold'), bg="#158FAD", fg="white", pady=20).pack(fill=X)

     # === Search and Date Filter Bar ===
    filter_frame = Frame(data_window, bg="#96C3EB")
    filter_frame.pack(pady=10, padx=20, fill=X)

    Label(filter_frame, text="Search Customer", font=('times new roman', 14), bg="#96C3EB").pack(side=LEFT)
    search_var = StringVar()
    search_entry = Entry(filter_frame, textvariable=search_var, font=('times new roman', 14), width=30)
    search_entry.pack(side=LEFT, padx=10)

    # Date range selection
    Label(filter_frame, text="From:", font=('times new roman', 14), bg="#96C3EB").pack(side=LEFT, padx=(20, 5))
    from_date = DateEntry(filter_frame, font=('times new roman', 14), date_pattern='yyyy-mm-dd')
    from_date.pack(side=LEFT)

    Label(filter_frame, text="To:", font=('times new roman', 14), bg="#96C3EB").pack(side=LEFT, padx=(20, 5))
    to_date = DateEntry(filter_frame, font=('times new roman', 14), date_pattern='yyyy-mm-dd')
    to_date.pack(side=LEFT)

    Button(filter_frame, text="Filter", font=('times new roman', 12), command=lambda: filter_table(), bg="#158FAD", fg="white"
          ).pack(side=LEFT, padx=20)

 
    # === Scrollable Canvas Setup ===
    canvas_frame = Frame(data_window)
    canvas_frame.pack(expand=True, fill=BOTH)

    canvas = Canvas(canvas_frame, bg="#96C3EB")
    canvas.pack(side=LEFT, fill=BOTH, expand=True)

    v_scroll = Scrollbar(canvas_frame, orient=VERTICAL, command=canvas.yview)
    v_scroll.pack(side=RIGHT, fill=Y)

    h_scroll = Scrollbar(data_window, orient=HORIZONTAL, command=canvas.xview)
    h_scroll.pack(side=BOTTOM, fill=X)
   
    canvas.configure(yscrollcommand=v_scroll.set, xscrollcommand=h_scroll.set)

    table_frame = Frame(canvas, bg="#EAF4FF")
    table_window= canvas.create_window((0, 0), window=table_frame, anchor="nw")

    def update_scrollregion(event=None):
        canvas.configure(scrollregion=canvas.bbox("all"))
        # Optional: Resize canvas window width
        canvas.itemconfig(table_window, width=canvas.winfo_width())

    table_frame.bind("<Configure>", update_scrollregion)
    canvas.bind("<Configure>", update_scrollregion)


    product_vars = {}
    product_names = {}

    # === Function to Populate the Table ===
    def render_table(filtered_data):
        for widget in table_frame.winfo_children():
            widget.destroy()

        headers = ["Select"] + columns
        for col_index, header in enumerate(headers):
            col_width = 6 if col_index == 0 else 18  # ✅ Narrow checkbox column
            Label(table_frame, text=header, font=('times new roman', 14, 'bold'), bg="#158FAD", fg="white", width=col_width, anchor="center"
                 ).grid(row=0, column=col_index, sticky="nsew", padx=2, pady=4)

        for row_index, row_data in enumerate(filtered_data):
            index = row_index
            var = IntVar()
            product_vars[index] = var
            product_names[index] = row_data[0]

            Checkbutton(table_frame, variable=var, bg="#EAF4FF").grid(row=row_index + 1, column=0, sticky="w", padx=5, pady=2)

            for col_index, cell_value in enumerate(row_data):
                Label(table_frame, text=str(cell_value), font=('times new roman', 12), bg="#EAF4FF",
                      anchor="center", justify=CENTER, width=15
                     ).grid(row=row_index + 1, column=col_index + 1, sticky="nsew", padx=2, pady=2)

    # === Filter Handler ===
    def filter_table(*args):
        query = search_var.get().strip().lower()
        from_d = from_date.get_date()
        to_d = to_date.get_date()

        def match(row):
            name_match = query in row[0].lower()
            try:
                row_date = datetime.strptime(row[3], '%Y-%m-%d').date()  # Assuming LastUpdated is at index 3
                date_match = from_d <= row_date <= to_d
            except Exception:
                date_match = False
            return name_match and date_match

        filtered = [row for row in data if match(row)]
        render_table(filtered)

    search_var.trace_add("write", filter_table)
    render_table(data)

    def _on_mousewheel(event):
        canvas.yview_scroll(int(-1*(event.delta/120)), "units")
        canvas.xview_scroll(int(-1*(event.delta/120)), "units") 
    canvas.bind("<MouseWheel>", _on_mousewheel)

    # === Process Button Action ===
    def update_selected_rows(new_status):
        selected_rows = [data[i] for i, v in product_vars.items() if v.get() == 1]
        if not selected_rows:
            messagebox.showwarning("No Selection", "Please select at least one quotation.")
            return

        try:
            for row in selected_rows:
                cursor.execute(
                    "UPDATE Quotations SET OrderStatus = ? WHERE CustomerName = ? AND ProductName = ? AND QuotationDate = ?",
                    (new_status, row[0], row[1], row[4]),
                )
            conn.commit()
            messagebox.showinfo("Success", f"Selected quotations marked as '{new_status}'.")
            # refresh data
            refreshed = fetch_customer_quotations()
            render_table(refreshed)
            update_graph([(r[1], r[2]) for r in refreshed], "Quotation Quantities", chart_type="bar")
        except Exception as exc:
            conn.rollback()
            messagebox.showerror("Error", str(exc))

    # === Action Buttons ===
    btn_frame = Frame(data_window, bg="#96C3EB")
    btn_frame.pack(pady=20)

    Button(btn_frame, text="Confirm", font=('times new roman', 15, 'bold'), bg="green", fg="white", width=12, command=lambda: update_selected_rows("Confirmed")
          ).pack(side=LEFT, padx=40)

    Button(btn_frame, text="Cancel", font=('times new roman', 15, 'bold'),bg="red", fg="white", width=12,command=lambda: update_selected_rows("Cancelled")
          ).pack(side=LEFT, padx=40)

    Button(btn_frame, text="Close", font=('times new roman', 15, 'bold'), command=data_window.destroy, width=12
          ).pack(side=LEFT, padx=40)

def fetch_customer_quotations():
    cursor.execute("SELECT CustomerName, ProductName, Quantity, Price, QuotationDate, OrderStatus FROM Quotations")
    return cursor.fetchall()

def open_quotation_window(_=None):
    rows = fetch_customer_quotations()
    show_data_in_product_window(
        "Customer Quotations",
        rows,
        ["CustomerName", "ProductName", "Quantity", "Price", "QuotationDate", "OrderStatus"],
    )
    update_graph([(r[1], r[2]) for r in rows], "Quotation Quantities", chart_type="bar")

product_frame=Frame(root, bg='#2C3E50', bd=3, relief=RIDGE)
product_frame.place(x=520, y=310, height=170, width=230)
total_product_image=PhotoImage(file='delivery-box.png')
total_product_image_label=Label(product_frame, image=total_product_image, bg='#2C3E50')
total_product_image_label.pack()
total_product_label=Label(product_frame, text='Quotation', font=('times new roman', 15, 'bold') ,bg='#2C3E50', fg='white')
total_product_label.pack()
total_product_count_label=Label(product_frame, text='0', bg='#2C3E50',fg='white', font=('times new roman', 20, 'bold'))
total_product_count_label.pack()
make_frame_clickable(product_frame, "Quotation",  open_quotation_window)

def fetch_order_status():
    cursor.execute("Select OrderID, CustomerID, ProductID, Quantity, TotalAmount, OrderDate, Status, PaymentMethod, ShippingAddress FROM Orders")
    return cursor.fetchall()

def open_order_window(title):
    order_data = fetch_order_status() 
    show_data_in_window(title, order_data, ["OrderID", "CustomerID", "ProductID", "Quantity", "TotalAmount", "OrderDate", "Status", "PaymentMethod", "ShippingAddress"])
    update_graph([(row[0], row[3]) for row in order_data], "Order Quantities", chart_type="bar")

order_frame=Frame(root, bg='#2C3E50', bd=3, relief=RIDGE)
order_frame.place(x=375, y=500, height=170, width=230)
total_order_image=PhotoImage(file='check-out.png')
total_order_image_label=Label(order_frame, image=total_order_image, bg='#2C3E50')
total_order_image_label.pack()
total_order_label=Label(order_frame, text="Logistics & Dispatch", font=('times new roman', 15, 'bold') ,bg='#2C3E50', fg='white')
total_order_label.pack()
total_order_count_label=Label(order_frame, text='0', bg='#2C3E50',fg='white', font=('times new roman', 20, 'bold'))
total_order_count_label.pack()
make_frame_clickable(order_frame, "Logistics & Dispatch", open_order_window)

root.mainloop()